In [ ]:
import os
# os.chdir('/kaggle/input/store-sales-time-series-forecasting/')
import numpy as np # linear algebra
import pandas as pd
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
# from statsmodels.tsa.statespace.sarimax import SARIMAX

# from statsmodels.graphics.tsaplots import plot_acf,plot_pacf # for determining (p,q) orders
# from statsmodels.tsa.seasonal import seasonal_decompose      # for ETS Plots
# from pmdarima import auto_arima                              # for determining ARIMA orders

# # Ignore harmless warnings
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
# ../input/store-sales-time-series-forecasting/holidays_events.csv

In [ ]:
holiday = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
transactions = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv',index_col='date',parse_dates=True)
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
train = pd.merge(train,transactions,left_on=['date','store_nbr'],right_on=['date','store_nbr'],how='left')
train['date'] = pd.to_datetime(train['date'])
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')
test = pd.merge(test,transactions,left_on=['date','store_nbr'],right_on=['date','store_nbr'],how='left')


In [ ]:
train['wages'] = ((train['date'].dt.day == 15)|(train['date'].dt.is_month_start)).map({True:1,False:0})
# train['Earth_quake'] = (train['date'].dt.strftime("%d %b %Y") == '16 Apr 2016').map({True:1,False:0})


holiday['date'] = pd.to_datetime(holiday['date'])
transfered = holiday[holiday['transferred'] == True].index
holiday.loc[transfered]['type'] = 'Work Day'
type_transfer = holiday[holiday['type'] == 'Transfer'].index
holiday.loc[type_transfer]['type'] = 'Holiday'

joined_df = pd.merge(train,stores[['store_nbr','cluster']],how='left',on='store_nbr')
joined_df['cluster'] == joined_df['cluster'].astype(str)
joined_df = joined_df.set_index('date')

holiday = holiday.set_index('date')
holiday['transferred'] = holiday['transferred'].map({True:1,False:0})
only_holiday = holiday[holiday['type'] == 'Holiday'][['type']]
only_holiday.columns = ['Holiday']
dummy_holidays = pd.get_dummies(holiday[['type']])
# dummy_holidays = pd.get_dummies(holiday)

joined_df = joined_df.join(dummy_holidays,how='left')

new_oil = pd.DataFrame(index=joined_df.index.unique(),data=oil)
new_oil = new_oil.interpolate(limit_direction='both')

joined_df = joined_df.join(new_oil,how='left')
joined_df = joined_df.fillna(value=0)

joined_df = joined_df.drop('id',axis=1)
joined_df = joined_df.drop('store_nbr',axis=1)
# joined_df['store_nbr'] = joined_df['store_nbr'].astype(str)
joined_df['cluster'] = joined_df['cluster'].astype(str)
joined_df = pd.get_dummies(joined_df)

joined_df['day'] = joined_df.index.day
joined_df['month'] = joined_df.index.month
joined_df['year'] = joined_df.index.year
# joined_df = joined_df.set_index('id')

In [ ]:
test['date'] = pd.to_datetime(test['date'])
test = pd.merge(test,stores[['store_nbr','cluster']],how='left',on='store_nbr')
test = test.merge(on='date',right=dummy_holidays,how='left')
new_oil_test = pd.DataFrame(index=test.date.unique(),data=oil)
new_oil_test = new_oil_test.reset_index()
new_oil_test.columns = ['date','dcoilwtico']
test['day'] = test.date.dt.day
test['month'] = test.date.dt.month
test['year'] = test.date.dt.year
test['wages'] = ((test['date'].dt.day == 15)|(test['date'].dt.is_month_start)).map({True:1,False:0})
# test['Earth_quake'] = (test['date'].dt.strftime("%d %b %Y") == '16 Apr 2016').map({True:1,False:0})
test = test.merge(new_oil_test,how='left',on='date')
test['dcoilwtico'] = test['dcoilwtico'].interpolate(limit_direction='both')
# test = test.set_index('id')
# test = test.drop('date',axis=1)
test = test.drop('id',axis=1)
test = test.set_index('date')
test = test.fillna(value=int(0))
test = test.drop('store_nbr',axis=1)
# test['store_nbr'] = test['store_nbr'].astype(str)
test['cluster'] = test['cluster'].astype(str)
test = pd.get_dummies(test)

In [ ]:
X = joined_df.drop('sales',axis=1)
y = joined_df['sales']

In [ ]:
# scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# scaled_X_train = scaler.fit_transform(X_train)
# scaled_X_test = scaler.transform(X_test)
# scaled_X = scaler.fit_transform(X)
# scaled_test = scaler.transform(test)

In [ ]:
# sgd = SGDRegressor(verbose=3,warm_start=True,eta0=0.1)
# ridge = Ridge()
# lin = LinearRegression()
eval_set = [(X_test, y_test)]
xgb = XGBRegressor(n_estimators=1000,eta=0.01,n_jobs=-1,verbosity=1,tree_method = 'gpu_hist')
# scaler = StandardScaler()

In [ ]:
xgb_model = xgb.fit(X_train,y_train,eval_metric="rmse", eval_set=eval_set,verbose=True,early_stopping_rounds=10,)
# xgb_model = xgb.fit(X,y,verbose=True)

In [ ]:
xgb.save_model('./1.json')

In [ ]:
y_hat = xgb_model.predict(test)

In [ ]:
pred = pd.DataFrame(data=y_hat,index=test.index)

In [ ]:
pred.to_csv('pred1.csv')